## Importing the relevant libraries

In [ ]:
!pip install wget

In [ ]:
pip install tensorflow==2.12.0

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings, wget, gc
warnings.filterwarnings('ignore')
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
wget.download("https://raw.githubusercontent.com/yogawicaksana/helper_prabowo/main/helper_prabowo_ml.py",out="helper_prabowo_ml.py")
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from wordcloud import WordCloud
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
from helper_prabowo_ml import clean_html, remove_, remove_digits, remove_digits, remove_special_characters, removeStopWords, remove_links, punct, email_address, lower, non_ascii

## Loading the dataset

In [ ]:
df = pd.read_csv("Symptom2Disease.csv")
df.drop('Unnamed: 0',axis=1,inplace=True)
df = df.sample(frac=0.5).reset_index(drop=True)
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()
df.shape

## Exploratory Data Analysis (EDA)

In [ ]:
plt.figure(figsize=(12,8))
wc = WordCloud(width=600,height=300,random_state=101).generate(' '.join(df.text))
plt.imshow(wc)
plt.title('Word Cloud of Medical Text',pad=20,fontsize=25,fontweight='bold',color='sienna');

In [ ]:
df['num_words'] = df.text.apply(len)
df.num_words.describe()

In [ ]:
plt.figure(figsize=(9,4))
fig = sns.distplot(df['num_words'])
plt.show(fig)
plt.close('all')
del fig
gc.collect()

In [ ]:
max_len = 170

In [ ]:
plt.figure(figsize=(16,10))
fig = sns.countplot(x='label',data=df)
plt.xticks(rotation=90,)
plt.tight_layout()
plt.show(fig)
plt.close('all')
del fig
gc.collect();

The target label classes i.e. the case outcomes are relatively imbalanced so this imbalance in the target labels needs to be accounted during model creation.

## Text Preprocessing

In [ ]:
def clean_text(data,col):
    data[col] = data[col].apply(func=clean_html)
    data[col] = data[col].apply(func=email_address)
    data[col] = data[col].apply(func=remove_)
    data[col] = data[col].apply(func=remove_digits)
    data[col] = data[col].apply(func=remove_links)
    data[col] = data[col].apply(func=remove_special_characters)
    data[col] = data[col].apply(func=removeStopWords)
    data[col] = data[col].apply(func=non_ascii)
    data[col] = data[col].apply(func=punct)
    data[col] = data[col].apply(func=lower)
    return data

In [ ]:
preprocessed_df = clean_text(df,'text')
preprocessed_df.head()

## Encoding the target label classes

In [ ]:
encoder = LabelEncoder()
preprocessed_df['label'] = encoder.fit_transform(preprocessed_df['label'])

In [ ]:
labels_dict = dict()

for idx, label in enumerate(encoder.classes_):
    labels_dict[idx] = label

labels_dict

## Splitting the preprocessed dataset into train and test sets

In [ ]:
train_df, test_df = train_test_split(preprocessed_df,test_size=0.3,stratify=preprocessed_df.label,shuffle=True,random_state=101)

## Downloading the Tokenizer and the pretrained BERT transformer model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('ajtamayoh/Symptoms_to_Diagnosis_SonatafyAI_BERT_v1')
bert_model = TFAutoModelForSequenceClassification.from_pretrained('ajtamayoh/Symptoms_to_Diagnosis_SonatafyAI_BERT_v1')

## Text Tokenization

In [ ]:
X_train = tokenizer(text=train_df.text.tolist(),
                   max_length=max_len,
                   padding='max_length',
                   truncation=True,
                   add_special_tokens=True,
                   return_attention_mask=True,
                   return_tensors='tf',
                   return_token_type_ids=False,
                   verbose=1)

X_test = tokenizer(text=test_df.text.tolist(),
                  max_length=max_len,
                  padding='max_length',
                  truncation=True,
                  add_special_tokens=True,
                  return_attention_mask=True,
                  return_tensors='tf',
                  return_token_type_ids=False,
                  verbose=1)

## Encoding the target labels

In [ ]:
y_train = to_categorical(train_df.label)
y_train

In [ ]:
y_test = to_categorical(test_df.label)
y_test

## Defining the model architecture

In [ ]:
input_ids = Input(shape=(max_len,),dtype=tf.int32,name='input_ids')
attention_mask = Input(shape=(max_len,),dtype=tf.int32,name='attention_mask')

In [ ]:
embeddings = bert_model(input_ids,attention_mask=attention_mask)[0] # 0 --> final hidden state, 1 --> pooling output
output = Flatten()(embeddings)
output = Dense(units=128,activation='relu')(output)
output = BatchNormalization()(output)
output = Dropout(0.2)(output)
output = Dense(units=64,activation='relu')(output)
output = BatchNormalization()(output)
output = Dropout(0.1)(output)
output = Dense(units=32,activation='relu')(output)
output = Dense(units=24,activation='softmax')(output)

model = Model(inputs=[input_ids,attention_mask],outputs=output)
model.layers[2].trainable = True
model.summary()

In [ ]:
plot_model(model,to_file='model.png',show_shapes=True,dpi=100)

## Compiling the model

In [ ]:
adam = Adam(learning_rate=5e-5, epsilon=2e-7, clipnorm=1.0)
model.compile(loss=CategoricalCrossentropy(from_logits=True),
              optimizer=adam,
              metrics=CategoricalAccuracy('balanced_accuracy'))

## Training the fine-tuned BERT transformer model

## Evaluating model performance on the test set

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder

# Load the data (assuming you have a pandas DataFrame named 'df' with 'text' and 'label' columns)
# If your data loading is different, adjust this part accordingly.
df = pd.read_csv('Symptom2Disease.csv') # I have created a dummy file with the same data.

# Separate text and labels
X = df['text']
y = df['label']

# Preprocess the text data
tokenizer = Tokenizer(num_words=10000)  # Limit vocabulary size
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=200)  # Pad sequences to a maximum length

# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_categorical, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=200))  # Embedding layer
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))  # LSTM layer
model.add(Dense(y_categorical.shape[1], activation='softmax'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Print model summary to show the architecture
print(model.summary())

# Set up callbacks
es = EarlyStopping(monitor='val_categorical_accuracy', patience=10, verbose=1, mode='max')
mc = ModelCheckpoint(filepath='symptoms_disease_classifier_lstm.keras',  #Different name for the file.
                     monitor='val_categorical_accuracy', mode='max', save_best_only=True, verbose=1)

# Train the model
history = model.fit(X_train, y_train,
                    epochs=50,  # Reduced number of epochs
                    batch_size=32,
                    validation_data=(X_test, y_test),
                    callbacks=[es, mc])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Load the best model
best_model = load_model('symptoms_disease_classifier_lstm.keras')

# Make predictions on the test set
y_pred = best_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Generate classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
print("Classification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_))

print("Confusion Matrix:")
cm = confusion_matrix(y_true_classes, y_pred_classes)
print(cm)


In [ ]:
import matplotlib.pyplot as plt

# Plot the training history
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], 'r', label='train loss')
plt.plot(history.history['val_loss'], 'b', label='test loss')
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')
plt.title('Loss Graph')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot the training history
plt.figure(figsize=(10, 5))
plt.plot(history.history['categorical_accuracy'], 'r', label='train balanced accuracy')
plt.plot(history.history['val_categorical_accuracy'], 'b', label='test balanced accuracy')
plt.xlabel('Number of Epochs')
plt.ylabel('Balanced Accuracy')
plt.title('Balanced Accuracy Graph')
plt.legend()
plt.show()

# Print the maximum balanced accuracy
max_train_accuracy = max(history.history['categorical_accuracy']) * 100
max_test_accuracy = max(history.history['val_categorical_accuracy']) * 100
print(f"Maximum Training Balanced Accuracy: {max_train_accuracy:.2f}%")
print(f"Maximum Testing Balanced Accuracy: {max_test_accuracy:.2f}%")
